In [1]:
println(s"Current spark version is ${spark.version}")

Intitializing Scala interpreter ...

Spark Web UI available at http://210a44f35d45:4040
SparkContext available as 'sc' (version = 3.0.0, master = local[*], app id = local-1598120955660)
SparkSession available as 'spark'


Current spark version is 3.0.0


In [2]:
import org.apache.spark.sql.types.{StructType, StructField, IntegerType, LongType, StringType}

val dataSchema = new StructType()
    .add("target", IntegerType)
    .add("id", LongType)
    .add("raw_timestamp", StringType)
    .add("query_status", StringType)
    .add("author", StringType)
    .add("tweet", StringType)

val dataPath= "/home/jovyan/data/training.1600000.processed.noemoticon.csv"

val raw_sentiment = spark.read
    .format("csv")
    .option("header",false)
    .schema(dataSchema)
    .load(dataPath)
    .selectExpr("(case when target=4 then 1 else 0 end) as label","tweet")

raw_sentiment.groupBy($"label").count.show

+-----+------+
|label| count|
+-----+------+
|    1|800000|
|    0|800000|
+-----+------+



import org.apache.spark.sql.types.{StructType, StructField, IntegerType, LongType, StringType}
dataSchema: org.apache.spark.sql.types.StructType = StructType(StructField(target,IntegerType,true), StructField(id,LongType,true), StructField(raw_timestamp,StringType,true), StructField(query_status,StringType,true), StructField(author,StringType,true), StructField(tweet,StringType,true))
dataPath: String = /home/jovyan/data/training.1600000.processed.noemoticon.csv
raw_sentiment: org.apache.spark.sql.DataFrame = [label: int, tweet: string]


In [3]:
//raw_sentiment.filter("label = 1").show(100, false)

In [4]:
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row

val tokenizer = new Tokenizer()
    .setInputCol("tweet")
    .setOutputCol("words")

val hashingTF = new HashingTF()
    .setNumFeatures(1000)
    .setInputCol(tokenizer.getOutputCol)
    .setOutputCol("features")

val lr = new LogisticRegression()
    .setMaxIter(10)
    .setRegParam(0.001)

val pipeline = new Pipeline()
  .setStages(Array(tokenizer, hashingTF, lr))


import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row
tokenizer: org.apache.spark.ml.feature.Tokenizer = tok_b70591323d67
hashingTF: org.apache.spark.ml.feature.HashingTF = HashingTF: uid=hashingTF_7e13d22faf0b, binary=false, numFeatures=1000
lr: org.apache.spark.ml.classification.LogisticRegression = logreg_9c751efd24f3
pipeline: org.apache.spark.ml.Pipeline = pipeline_ee055b041b74


In [5]:
val model = pipeline.fit(raw_sentiment)

model: org.apache.spark.ml.PipelineModel = pipeline_ee055b041b74


In [6]:
model.write.overwrite().save("/home/jovyan/models/spark-ml-model")

In [7]:
val sameModel = PipelineModel.load("/home/jovyan/models/spark-ml-model")

sameModel: org.apache.spark.ml.PipelineModel = pipeline_ee055b041b74


In [8]:
val predictionsDF = sameModel.transform(raw_sentiment)

predictionsDF.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|               tweet|               words|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    0|@switchfoot http:...|[@switchfoot, htt...|(1000,[10,21,81,1...|[-1.7093077824651...|[0.15325352111540...|       1.0|
|    0|is upset that he ...|[is, upset, that,...|(1000,[121,193,20...|[0.98452561467198...|[0.72800527368281...|       0.0|
|    0|@Kenichan I dived...|[@kenichan, i, di...|(1000,[17,185,188...|[-0.2839685001973...|[0.42948111509142...|       1.0|
|    0|my whole body fee...|[my, whole, body,...|(1000,[191,330,44...|[0.15558576510072...|[0.53881816712716...|       0.0|
|    0|@nationwideclass ...|[@nationwideclass...|(1000,[32,162,166...|[2.91879438070464...|[0.94876772860819...|       0.0|
|    0|@

predictionsDF: org.apache.spark.sql.DataFrame = [label: int, tweet: string ... 5 more fields]


In [12]:
predictionsDF.filter("label = 1").show(100)

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|               tweet|               words|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    1|I LOVE @Health4Ua...|[i, love, @health...|(1000,[17,216,240...|[-1.2100795585179...|[0.22968697429303...|       1.0|
|    1|im meeting up wit...|[im, meeting, up,...|(1000,[20,26,29,1...|[0.65024959779721...|[0.65706670676220...|       0.0|
|    1|@DaRealSunisaKim ...|[@darealsunisakim...|(1000,[10,17,19,2...|[-1.3618052335367...|[0.20394706118063...|       1.0|
|    1|Being sick can be...|[being, sick, can...|(1000,[73,76,112,...|[1.61213051745973...|[0.83370697067522...|       0.0|
|    1|@LovesBrooklyn2 h...|[@lovesbrooklyn2,...|(1000,[66,324,559...|[-0.2714939166046...|[0.43254037751081...|       1.0|
|    1|@

In [8]:
// predictionsDF.schema
// predictionsDF.describe()
// predictionsDF.stat
predictionsDF.printSchema()

root
 |-- label: integer (nullable = false)
 |-- tweet: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [13]:
import org.apache.spark.sql.functions._

val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))

import org.apache.spark.sql.functions._
getProbability: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$4580/0x0000000841927040@740a4059,DoubleType,List(Some(class[value[0]: vector])),None,false,true)


In [ ]:
predictionsDF.select(getProbability($"probability").alias("clean_probability")).show